In [2]:
import torch
from ettcl.core.pipelines import ClassificationPipeline
from datasets import load_from_disk, Split
from sklearn.metrics import accuracy_score

text_column = 'text'
label_column = 'label'

In [58]:
dataset = load_from_disk("/home/IAIS/hiser/data/trec-50")
dataset.set_format('torch')

train_dataset = dataset[Split.TRAIN]
test_dataset = dataset[Split.TEST].select(range(500))

label_names = train_dataset.features[label_column].names

In [60]:
pipeline = ClassificationPipeline(
    "/home/IAIS/hiser/dev/hyped/examples/output/trec_50_bert/model/best-model",
    architecture='colbert',
    # tokenizer_kwargs={'query_maxlen': 16, 'doc_maxlen': 16},
    tokenizer_kwargs={'query_token': None, 'doc_token': None, 'add_special_tokens': False},
    # indexer_kwargs={'num_partitions': 2048},
    searcher_kwargs={'ncells': 16},
    # searcher_kwargs={'ncells': 1},
    k=5,
    # num_sentences=32,
)
pipeline.train_index(train_dataset)
pipeline.train_index(train_dataset, index_path="outputs/index_colbert")

Some weights of the model checkpoint at /home/IAIS/hiser/dev/hyped/examples/output/trec_50_bert/model/best-model were not used when initializing ColBERTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing ColBERTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
#> Will delete 14 files already at outputs/index_colbert
[0][2023-07-27 17:10:04,588] [INFO] [init] nranks = 2 	 gpus = [0, 1] 	 device=0
[1][2023-07-27 17:10:08,634] [INFO] [init] nranks = 2 	 gpus = [0, 1] 	 device=1
Encoding: 100%|██████████| 10/10 [00:01<00:00,  7.99it/s]
[0][2023-07-27 17:10:11,514] [INFO] [print_message] [0] 		 avg_d

Clustering 53470 points in 768D to 2048 clusters, redo 1 times, 4 iterations
  Preprocessing in 0.04 s
  Iteration 3 (0.17 s, search 0.14 s): objective=5969.54 imbalance=1.478 nsplit=0       


[0][2023-07-27 17:10:14,797] [INFO] [print_message] Loading decompress_residuals_cuda_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[0][2023-07-27 17:10:14,933] [INFO] [print_message] Loading packbits_cuda_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[0][2023-07-27 17:10:15,067] [INFO] [print_message] #> Got bucket_cutoffs_quantiles = tensor([0.2500, 0.5000, 0.7500], device='cuda:0') and bucket_weights_quantiles = tensor([0.1250, 0.3750, 0.6250, 0.8750], device='cuda:0')
[0][2023-07-27 17:10:15,069] [INFO] [print_message] #> Got bucket_cutoffs = tensor([-7.5989e-03,  6.1035e-05,  7.7209e-03], device='cuda:0') and bucket_weights = tensor([-0.0137, -0.0035,  0.0036,  0.0138], device='cuda:0')
[0][2023-07-27 17:10:15,069] [INFO] [print_message] avg_residual = 0.0097808837890625
[1][2023-07-27 17:10:15,112] [INFO] [print_message] Loading decompress_residuals_cuda_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True 

In [63]:
pipeline.k = 10
preds = pipeline.predict(test_dataset[text_column], test_dataset[label_column])
accuracy_score(test_dataset[label_column], preds)

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Searching (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

0.81

In [64]:
torch.where(preds != test_dataset[label_column])[0]

tensor([  1,   8,  15,  30,  31,  33,  34,  44,  45,  58,  59,  68,  81,  84,
         85,  92,  99, 104, 106, 116, 117, 119, 121, 124, 133, 135, 138, 143,
        151, 153, 158, 172, 176, 178, 189, 193, 196, 200, 203, 207, 224, 234,
        239, 241, 249, 251, 252, 259, 280, 291, 292, 296, 298, 325, 326, 327,
        346, 356, 357, 358, 362, 363, 365, 367, 369, 371, 376, 382, 384, 393,
        398, 402, 404, 407, 420, 422, 425, 432, 441, 443, 450, 452, 454, 455,
        457, 460, 467, 474, 480, 481, 484, 487, 488, 494, 498])

In [91]:
maxsim_view = pipeline.explain_prediction(15, 0)
maxsim_view

Query 15: What is the life expectancy for crickets ?
True Label: NUM:other, predicted label: NUM:period
Match 0: What is the life expectancy of an elephant ?


: 